In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from typing import List, Optional, AsyncGenerator
from datetime import datetime
from pydantic import Field
from core import BaseCall, Msg
import instructor
from openai import AsyncOpenAI
import pandas as pd
from typing import Dict
from sampling import WeightedSampler, fetch_joined_data
from query_planner import QueryPlan
import enum
from typing import List
from pydantic import Field
from core import user_msg

from core import system_msg,user_msg,assistant_msg

from IPython.display import display,clear_output

In [5]:
from pydantic_models import EntityExtraction
from pydantic import field_validator

In [6]:
from sampling import fetch_joined_data,WeightedSampler

df = await fetch_joined_data()
df = df.sort_values(by='meeting_timestamp')

In [21]:
class ConsolidatedNode(BaseCall):
    ids: List[int] = Field(..., description="List of indices that should be consolidated into this node, at least 2")
    topic: str = Field(..., description="Main topic name for this consolidated group")

    @field_validator('ids')
    def validate_ids(cls, v):
        if len(v) < 1:
            raise ValueError("Each consolidated node must contain at least 2 related items")
        return v

class KnowledgeGraph(BaseCall):
    nodes: List[ConsolidatedNode] = Field(..., description="List of consolidated topic nodes")


df_sample = df.sample(200).reset_index()


In [22]:
messages = [
    system_msg(f"""You are an expert knowledge graph builder. Analyze the meeting data and consolidate topics into business-focused categories, following these rules:

1. NATURAL GROUPING:
   - Group size should reflect ACTUAL relationships
   - Some topics may have many related items (6-10+)
   - Others may have just 2-3 closely related items
   - DO NOT try to balance group sizes
   - Let the data guide the grouping

2. BUSINESS CATEGORIES:
   - People & Organizations
     * Team members, customers, partners, competitors
   - Products & Services 
     * Our offerings, competitor products, tools used
   - Projects & Initiatives
     * Development, marketing, sales efforts
   - Tasks & Actions
     * Specific work items and next steps

3. VALIDATION:
   - Minimum 2 related items per group
   - Must have clear business context
   - Must be specific (e.g., "Google Ads Q4 Campaign" not just "Marketing")
   - No mixing unrelated items just to fill groups

4. AVOID:
   - Creating equal-sized groups
   - Using sequential IDs
   - Generic topic names
   - Mixing different categories
   - Forcing items into groups

Meeting data: {df_sample[['index', 'summary', 'speaker_name', 'topic_name']].to_csv()}"""),
    user_msg("Create consolidated nodes based on ACTUAL topic relationships in the data. Group sizes should vary naturally - some topics may have many related items, others just a few closely related ones. Focus on real connections, not balanced groups.")
]

In [28]:
r = await KnowledgeGraph.call(messages)


In [29]:
r.model_dump()

{'nodes': [{'ids': [0, 19, 85], 'topic': 'AI Technologies and Applications'},
  {'ids': [1, 10, 38, 146], 'topic': 'Customer Insights and Market Research'},
  {'ids': [3, 139, 35, 4, 59], 'topic': 'Cloud Services and Migration'},
  {'ids': [6, 9, 34, 59, 173], 'topic': 'Transcription and Meeting Tools'},
  {'ids': [2, 25, 158], 'topic': 'Programming Languages'},
  {'ids': [11, 23, 74, 191], 'topic': 'User Interface and Experience'},
  {'ids': [12, 50, 60, 192], 'topic': 'System Functionality and Testing'},
  {'ids': [14, 41, 192, 123], 'topic': 'Project Management and Collaboration'},
  {'ids': [7, 20, 114, 186], 'topic': 'Legal and Compliance Considerations'},
  {'ids': [8, 45, 96, 145], 'topic': 'Meeting Organization and Follow-ups'},
  {'ids': [13, 52, 113, 198], 'topic': 'Startup and Business Development'},
  {'ids': [18, 31, 67, 189], 'topic': 'Social Media and Marketing Strategies'},
  {'ids': [33, 37, 59, 59], 'topic': 'SaaS and Software Development'},
  {'ids': [32, 70, 82, 91]

In [32]:
grouped_dfs = {}

# For each node in the consolidated results
for node in r.nodes:
    # Get the topic name
    topic = node.topic
    # Filter ids to only include those that exist in df_sample
    valid_ids = [id for id in node.ids if id in df_sample.index]
    if valid_ids:  # Only create group if there are valid IDs
        # Get the corresponding rows from df_sample using the filtered ids
        group_df = df_sample.loc[valid_ids, ['index', 'topic_name', 'summary']]
        grouped_dfs[topic] = group_df

for topic, group_df in grouped_dfs.items():
    print(f"\n=== {topic} ===")
    display(group_df)



=== AI Technologies and Applications ===


,index,topic_name,summary
0,2179,AI assistance in user interviews,The potential for AI to assist in user intervi...
19,2147,AI,AI refers to artificial intelligence technolog...
85,1037,AI,AI (Artificial Intelligence) is a central topi...



=== Customer Insights and Market Research ===


,index,topic_name,summary
1,1545,customer base,Customer base refers to the group of clients t...
10,1443,Market Research,The team plans to conduct market research to b...
38,2107,B2C,B2C refers to business-to-consumer marketing s...
146,2615,Analyze Google Ads usage among competitors,Olga reviewed the Google Ads usage of competit...



=== Cloud Services and Migration ===


...

,index,topic_name,summary
189,586,social media,Social media platforms are used for marketing ...
190,668,Andy,Andy is mentioned as an influential figure who...
191,1321,dashboard,"The dashboard is a key focus of the meeting, w..."
192,2911,Implement backend processing for artifact storage,Backend systems need to be developed to manage...



=== Technical Support and Resources ===


,index,topic_name,summary
193,1884,FastAPI,FastAPI is a modern web framework for building...
194,2874,summary,Summary refers to the automatic processing of ...
195,2561,Understanding the functionality of highlights,There is a need to clarify how the highlightin...
196,1290,Кейнборды,Кейнборды are discussed as part of the marketi...



=== Market Research and Analysis ===


,index,topic_name,summary
197,1111,mom,"A personal reference made during the meeting, ..."
198,2321,Discussion on Product Hunt and its significance,The participants discussed the potential of la...
199,2668,Siberia,Siberia is the region where the factory using ...


In [12]:
output_df = pd.DataFrame(r.model_dump()['consolidated_items'])


In [13]:
output_df

""


In [100]:
pd.options.display.max_colwidth = 1000

In [101]:
output_df

,topic,topic_type,summary,details,related_topics
0,Telegram,platform,"Telegram is a cloud-based instant messaging service that allows users to send messages, photos, and videos. It is being considered for marketing outreach, but there are concerns about its future due to uncertainties surrounding its ownership and moderation policies in different regions, particularly in the Emirates and Saudi Arabia. The team plans to test Telegram's effectiveness in the Emirates, where it reportedly has a good audience and moderation.","- Telegram is a messaging platform being considered for marketing outreach. - Concerns were raised about the future of Telegram due to uncertainties surrounding its ownership and moderation policies. - Testing is planned in the Emirates, where it is believed to have a good audience.","[Marketing Strategy, User Engagement]"
1,Google Ads,platform,"Google Ads is an online advertising platform that the team plans to utilize for tracking and promoting their content. The campaign is set to launch on Monday after being paused for the weekend, with expectations to drive traffic to the site and increase user registrations. The team is also focusing on adding mobile features to enhance advertising capabilities.",- Google Ads is a platform for advertising that the team plans to utilize for tracking and promoting their content. - The campaign is set to launch on Monday after being paused for the weekend. - The team is focusing on adding mobile features to enhance advertising capabilities.,"[Marketing Strategy, User Acquisition]"
2,YouTube,platform,"YouTube is a video-sharing platform that the team is focusing on for content creation and distribution. The team plans to publish multiple videos, including tutorials and promotional content, to enhance visibility and engagement. They aim to analyze user interaction data on the platform to inform future strategies.","- YouTube is a platform for video content creation and distribution. - The team plans to publish multiple videos, including tutorials and promotional content. - They aim to analyze user interaction data to inform future strategies.","[Content Strategy, User Engagement]"
3,User Engagement,concept,User engagement refers to the interaction and involvement of users with VEX. The team is monitoring engagement metrics from posts to understand audience interaction and is exploring strategies to enhance user retention and acquisition through various marketing channels.,- User engagement is crucial for the success of marketing efforts. - The team is monitoring engagement metrics from posts to understand audience interaction. - Strategies are being explored to enhance user retention and acquisition.,"[User Retention, Marketing Metrics]"
4,Influencer Marketing,strategy,Influencer marketing is a strategy discussed for promoting the product through collaborations with social media influencers. The team is actively reaching out to influencers on platforms like TikTok and Instagram to enhance brand visibility and engagement.,- Influencer marketing is a strategy for promoting the product. - The team is reaching out to influencers on TikTok and Instagram. - Collaborations with influencers are expected to enhance brand visibility and engagement.,"[Social Media Marketing, Community Building]"
5,Community Building,concept,"Community building involves creating a space for users to interact and share experiences. The team discussed the need to establish a community around the product to enhance user engagement and retention, utilizing platforms like WhatsApp and Telegram for communication.",- Community building is essential for user engagement and retention. - The team discussed establishing a community around the product. - Platforms like WhatsApp and Telegram are considered for communication.,"[User Engagement, Social Media]"
6,VEX,product,VEX is a product being developed that focuses on providing real-time transcription and contextual support during meetings. The team is working on 

In [87]:
df[df['topic_name']=='Start TikTok marketing in Brazil']



,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers
1368,2,Proposed to test TikTok marketing in Brazil due to its active audience and potential for growth.,"Olga suggested Brazil as a target market for TikTok due to its developing status and active audience. The content will be in Brazilian Portuguese. No specific timeline was mentioned, but it is part of the marketing strategy.","Olga Nemirovskaya: Короче, TikTok, сделаю я TikTok нам аккаунт, и мы его запустим на Бразилию, потому что нам там надо выбрать одну страну. | Olga Nemirovskaya: И решить, где мы потестим. | Olga Nemirovskaya: Я предлагаю Бразилию потестить тоже на ТикТоке. | Olga Nemirovskaya: Или у тебя есть какие-нибудь предложения лучше? | Olga Nemirovskaya: Почему Бразилию? | Olga Nemirovskaya: Потому что Google Meet активный, потому что достаточно активная аудитория, нам лучше брать страну развивающуюся. | Olga Nemirovskaya: но не самую как бы бедную и потому что трейла тоже очень активные как будто бы мы там с Совпадаем. А, и еще вот тоже, когда с Ларой я общалась, она говорит, что там и DHD комьюнити тоже очень сильное в Бразилии. Ну, в общем, то есть как будто бы это...",Start TikTok marketing in Brazil,future projection,99ade764-fd60-4e9a-9621-66cc037cb865,2024-08-27 19:34:48.996,Olga Nemirovskaya,[Dmitriy Grankin]
